#### 필요 패키지

In [ ]:
import requests # type: ignore
import uuid
import gradio as gr # type: ignore
import time
from __future__ import annotations
from typing import Iterable
import time
import re
import os
from dotenv import load_dotenv

load_dotenv()
STEAM_ENDPOINT = os.getenv("STEAM_ENDPOINT")

SPEECH_ENDPOINT_STEAM = os.getenv("SPEECH_ENDPOINT_STEAM")
SPEECH_API_KEY_STEAM = os.getenv("SPEECH_API_KEY_STEAM")
SPEECH_SUMMARIZATION_ENDPOINT_STEAM = os.getenv("SPEECH_SUMMARIZATION_ENDPOINT_STEAM")

OPENAI_ENDPOINT_STEAM = os.getenv("OPENAI_ENDPOINT_STEAM")
OPENAI_APIKEY_STEAM = os.getenv("OPENAI_APIKEY_STEAM")

TTS_ENDPOINT_STEAM = os.getenv("TTS_ENDPOINT_STEAM")
TTS_APIKEY_STEAM = os.getenv("TTS_APIKEY_STEAM")

### 개별 기능(함수) 구현 과정

#### Steam에서 리뷰 가져오기
스팀에 있는 게임 이름을 검색하면 그 이름을 가지고 id를 뽑아서 영어로 된 리뷰를 가져옴

In [ ]:
def get_exact_app_id(game_name):
    url = STEAM_ENDPOINT
    response = requests.get(url)
    data = response.json()

    for app in data["applist"]["apps"]:
        if app["name"].strip().lower() == game_name.strip().lower():  # 완전 일치 검사
            return app["appid"]
    return None


game_title = "ELDEN RING"  # 정확한 게임 제목 입력
app_id = get_exact_app_id(game_title)

APP_ID = app_id  # 원하는 게임의 Steam App ID
URL = f"https://store.steampowered.com/appreviews/{APP_ID}?json=1&language=english"

response = requests.get(URL)
reviews = response.json()

print(reviews['reviews'][0]['review'])


Elden Ring is the first FromSoftware game that i actually played and i enjoyed it a lot but its not a perfect game by any means, I would rate it a strong 8 out of 10. There are a lot of things the game does right and well but it has its short comings too

Pros;
Really good boss designs
Art style of the game fits it very well and looks really good
A lot of different viable builds and playstyles
A Big map thats actually filled with content and is not empty
There are some really good quest lines if you follow and pay attention to them
Progression system most of the time feels rewarding

Cons;
Game repeats itself too much, you fight the same bosses over and over again, dungeon layouts are almost identical with only difference being their gimmick or the enemies inside of it
There are a lot of weapons and spells in the game but a lot of them are not viable at all or different versions of the same thing
Game practically forces you to play it at new game+ and finish it again if you want to get

#### 감정 분석 수행

In [ ]:
def get_exact_app_id(game_name):
    url = STEAM_ENDPOINT
    response = requests.get(url)
    data = response.json()

    for app in data["applist"]["apps"]:
        if app["name"].strip().lower() == game_name.strip().lower():  # 완전 일치 검사
            return app["appid"]
    return None

def request_SentimentAnalysis(text="", language="en"):
    endpoint = SPEECH_ENDPOINT_STEAM
    header = {
        "Content-Type":"application/json",
        "Ocp-Apim-Subscription-Key":SPEECH_API_KEY_STEAM
    }

    request_id = str(uuid.uuid4())

    body = {
        "kind": "SentimentAnalysis",
        "parameters": {
            "modelVersion": "latest",
            "opinionMining": "True"
        },
        "analysisInput": {
            "documents": [
                {
                    "id": request_id,
                    "language": language,
                    "text": text
                }
            ]
        }
    }

    response = requests.post(endpoint, headers=header, json=body)
    if response.status_code == 200:
        response_json = response.json()
        return response_json
    else:
        return None


def request_Summarization(text, language="en"):
    endpoint = SPEECH_SUMMARIZATION_ENDPOINT_STEAM
    headers = {
        "Content-Type": "application/json",
        "Ocp-Apim-Subscription-Key": SPEECH_API_KEY_STEAM
    }

    request_id = str(uuid.uuid4())
    
    body = {
        "displayName": "Text Summarization Task",
        "analysisInput": {
            "documents": [
                {
                    "id": request_id,
                    "language": language,
                    "text": text
                }
            ]
        },
        "tasks": [
            {
                "kind": "ExtractiveSummarization",
                "taskName": "Text Extractive Summarization Task 1",
                "parameters": {
                    "sentenceCount": 15
                }
            }
        ]
    }

    response = requests.post(endpoint, headers=headers, json=body)

    if response.status_code == 202:
        result_url = response.headers['Operation-Location']

        while True:
            result_response = requests.get(result_url, headers=headers)
            result_data = result_response.json()
            result_status = result_data.get("status", "")

            # 응답 상태 확인
            if result_status == "succeeded":
                break
            elif result_status in ["failed", "canceled"]:
                return None
            time.sleep(2)  # 2초 대기 후 다시 요청

        try:
            # 요약된 텍스트 문장 리스트 가져오기
            sentences = result_data['tasks']['items'][0]['results']['documents'][0]['sentences']
            summarized_text = " ".join([sentence['text'] for sentence in sentences])
            return summarized_text
        except KeyError:
            return None
    else:
        return None


game_title = "ELDEN RING"  # 정확한 게임 제목 입력
app_id = get_exact_app_id(game_title)

APP_ID = app_id  # 원하는 게임의 Steam App ID
URL = f"https://store.steampowered.com/appreviews/{APP_ID}?json=1&language=english"

response = requests.get(URL)
reviews = response.json()

# 모든 리뷰 텍스트 합치기
all_reviews_text = " ".join([review['review'] for review in reviews['reviews']])

# 전체 리뷰에 대해 요약 요청
summarized_reviews = request_Summarization(all_reviews_text)

if summarized_reviews:
    # 요약된 리뷰에 대해 감정 분석 요청
    sentiment_analysis_result = request_SentimentAnalysis(text=summarized_reviews)
    print(sentiment_analysis_result)
else:
    print("Error: Summary not available.")


{'kind': 'SentimentAnalysisResults', 'results': {'documents': [{'id': 'db51fc9e-3693-4de7-b277-5aafa4f88d8c', 'sentiment': 'mixed', 'confidenceScores': {'positive': 0.83, 'neutral': 0.01, 'negative': 0.16}, 'sentences': [{'sentiment': 'negative', 'confidenceScores': {'positive': 0.24, 'neutral': 0.02, 'negative': 0.74}, 'offset': 0, 'length': 167, 'text': 'Elden Ring is the first FromSoftware game that i actually played and i enjoyed it a lot but its not a perfect game by any means, I would rate it a strong 8 out of 10. ', 'targets': [{'sentiment': 'positive', 'confidenceScores': {'positive': 1.0, 'negative': 0.0}, 'offset': 0, 'length': 10, 'text': 'Elden Ring', 'relations': [{'relationType': 'assessment', 'ref': '#/documents/0/sentences/0/assessments/0'}]}, {'sentiment': 'positive', 'confidenceScores': {'positive': 1.0, 'negative': 0.0}, 'offset': 24, 'length': 17, 'text': 'FromSoftware game', 'relations': [{'relationType': 'assessment', 'ref': '#/documents/0/sentences/0/assessments/

#### 감정 분석한 거 바탕으로 open ai 돌려서 결과 받기

In [ ]:
# 스팀에서 게임 ID 가져오기
def get_exact_app_id(game_name):
    url = STEAM_ENDPOINT
    response = requests.get(url)
    data = response.json()

    for app in data["applist"]["apps"]:
        if app["name"].strip().lower() == game_name.strip().lower():
            return app["appid"]
    return None

# 감정 분석 요청
def request_SentimentAnalysis(text="", language="en"):
    endpoint = SPEECH_ENDPOINT_STEAM
    header = {
        "Content-Type":"application/json",
        "Ocp-Apim-Subscription-Key":SPEECH_API_KEY_STEAM
    }

    request_id = str(uuid.uuid4())

    body = {
        "kind": "SentimentAnalysis",
        "parameters": {
            "modelVersion": "latest",
            "opinionMining": "True"
        },
        "analysisInput": {
            "documents": [
                {
                    "id": request_id,
                    "language": language,
                    "text": text
                }
            ]
        }
    }

    response = requests.post(endpoint, headers=header, json=body)
    if response.status_code == 200:
        response_json = response.json()
        return response_json
    else:
        return None

# 텍스트 요약 요청
def request_Summarization(text, language="en"):
    endpoint = SPEECH_SUMMARIZATION_ENDPOINT_STEAM
    headers = {
        "Content-Type": "application/json",
        "Ocp-Apim-Subscription-Key": SPEECH_API_KEY_STEAM
    }

    request_id = str(uuid.uuid4())
    
    body = {
        "displayName": "Text Summarization Task",
        "analysisInput": {
            "documents": [
                {
                    "id": request_id,
                    "language": language,
                    "text": text
                }
            ]
        },
        "tasks": [
            {
                "kind": "ExtractiveSummarization",
                "taskName": "Text Extractive Summarization Task 1",
                "parameters": {
                    "sentenceCount": 15
                }
            }
        ]
    }

    response = requests.post(endpoint, headers=headers, json=body)

    if response.status_code == 202:
        result_url = response.headers['Operation-Location']

        while True:
            result_response = requests.get(result_url, headers=headers)
            result_data = result_response.json()
            result_status = result_data.get("status", "")

            if result_status == "succeeded":
                break
            elif result_status in ["failed", "canceled"]:
                return None
            time.sleep(2)

        try:
            sentences = result_data['tasks']['items'][0]['results']['documents'][0]['sentences']
            summarized_text = " ".join([sentence['text'] for sentence in sentences])
            return summarized_text
        except KeyError:
            return None
    else:
        return None

# gpt-openai 챗봇
def request_gpt(prompt):
    endpoint = OPENAI_ENDPOINT_STEAM
    headers = {
        "Content-Type": "application/json",
        "api-key": OPENAI_APIKEY_STEAM
    }

    body = {
        "messages": [
            {
                "role": "system",
                "content": [
                    {
                        "type": "text",
                        "text": "너는 나를 도와주는 도우미야."
                    }
                ]
            },
            {
                "role": "user",
                "content": [
                    {
                        "type": "text",
                        "text": prompt
                    }
                ]
            }
        ],
        "temperature": 0.7,
        "top_p": 0.9,
        "max_tokens": 800
    }

    response = requests.post(endpoint, headers=headers, json=body)
    if response.status_code == 200:        
        response_json = response.json()
        message = response_json['choices'][0]['message']
        content = message['content']
        return content
    else:
        return ""

# 전체 리뷰 요약 및 감정 분석 후 GPT에게 게임 추천 여부 묻기
def game_recommendation(game_name):
    app_id = get_exact_app_id(game_name)
    if app_id is None:
        return "게임을 찾을 수 없습니다."

    URL = f"https://store.steampowered.com/appreviews/{app_id}?json=1&language=english"
    response = requests.get(URL)
    reviews = response.json()

    all_reviews_text = " ".join([review['review'] for review in reviews['reviews']])

    summarized_reviews = request_Summarization(all_reviews_text)
    if summarized_reviews:
        sentiment_analysis_result = request_SentimentAnalysis(text=summarized_reviews)
        if sentiment_analysis_result:
            # 챗봇에게 감정 분석 결과 전달하여 게임 추천 여부 물어보기
            prompt = f"이 게임의 감정 분석 결과는 {sentiment_analysis_result}. 이 게임을 추천할까요?"
            chatbot_response = request_gpt(prompt)
            return chatbot_response
        else:
            return "감정 분석 결과를 가져올 수 없습니다."
    else:
        return "리뷰 요약에 실패했습니다."

# 예시: 게임 이름을 입력받아 추천 여부를 받음
game_name = "Human Fall Flat"
recommendation = game_recommendation(game_name)
print(recommendation)


### 실제 구현 코드

#### 테마 설정

In [7]:
theme = gr.themes.Citrus(primary_hue="sky", secondary_hue="teal", neutral_hue="blue").set(
        block_title_text_weight="600",
        block_border_width="3px",
        block_shadow="*shadow_drop_lg",
        button_primary_shadow="*shadow_drop_lg",
        button_large_padding="32px",
)

custom_css: str = """ 
    .gradio-container {
    background: url('https://img.freepik.com/free-vector/earth-view-night-from-alien-planet-neon-space_33099-1876.jpg?t=st=1742540483~exp=1742544083~hmac=be2d0f807c478dca31559bca2a3d9a9928cb93dbd9c817a628776f57e266522a&w=1380');
    background-size: cover;
    padding-left: 120px !important;
    padding-right: 120px !important;
    }" 
"""


#### 구현코드

In [ ]:
# 스팀에서 게임 ID 가져오기
def get_exact_app_id(game_name):
    url = STEAM_ENDPOINT
    response = requests.get(url)
    data = response.json()

    for app in data["applist"]["apps"]:
        if app["name"].strip().lower() == game_name.strip().lower():
            return app["appid"]
    return None

# 감정 분석 요청
def request_SentimentAnalysis(text="", language="en"):
    endpoint = SPEECH_ENDPOINT_STEAM
    header = {
        "Content-Type":"application/json",
        "Ocp-Apim-Subscription-Key":SPEECH_API_KEY_STEAM
    }

    request_id = str(uuid.uuid4())

    body = {
        "kind": "SentimentAnalysis",
        "parameters": {
            "modelVersion": "latest",
            "opinionMining": "True"
        },
        "analysisInput": {
            "documents": [
                {
                    "id": request_id,
                    "language": language,
                    "text": text
                }
            ]
        }
    }

    response = requests.post(endpoint, headers=header, json=body)
    if response.status_code == 200:
        response_json = response.json()
        return response_json
    else:
        return None

# 텍스트 요약 요청
def request_Summarization(text, language="en"):
    endpoint = SPEECH_SUMMARIZATION_ENDPOINT_STEAM
    headers = {
        "Content-Type": "application/json",
        "Ocp-Apim-Subscription-Key": SPEECH_API_KEY_STEAM
    }

    request_id = str(uuid.uuid4())
    
    body = {
        "displayName": "Text Summarization Task",
        "analysisInput": {
            "documents": [
                {
                    "id": request_id,
                    "language": language,
                    "text": text
                }
            ]
        },
        "tasks": [
            {
                "kind": "ExtractiveSummarization",
                "taskName": "Text Extractive Summarization Task 1",
                "parameters": {
                    "sentenceCount": 15
                }
            }
        ]
    }

    response = requests.post(endpoint, headers=headers, json=body)

    if response.status_code == 202:
        result_url = response.headers['Operation-Location']

        while True:
            result_response = requests.get(result_url, headers=headers)
            result_data = result_response.json()
            result_status = result_data.get("status", "")

            if result_status == "succeeded":
                break
            elif result_status in ["failed", "canceled"]:
                return None
            time.sleep(2)

        try:
            sentences = result_data['tasks']['items'][0]['results']['documents'][0]['sentences']
            summarized_text = " ".join([sentence['text'] for sentence in sentences])
            return summarized_text
        except KeyError:
            return None
    else:
        return None
    
# GPT 응답 요청 함수
def request_gpt(prompt):
    endpoint = OPENAI_ENDPOINT_STEAM
    headers = {
        "Content-Type": "application/json",
        "api-key": OPENAI_APIKEY_STEAM
    }

    body = {
        "messages": [
            {
                "role": "system",
            "content": [
                    {
                        "type": "text",
                        "text": "너는 나를 도와주는 도우미야."
                    }
                ]
            },
            {
                "role": "user",
                "content": [
                    {
                        "type": "text",
                        "text": prompt
                    }
                ]
            }
        ],
        "temperature": 0.7,
        "top_p": 0.9,
        "max_tokens": 800
    }

    response = requests.post(endpoint, headers=headers, json=body)
    if response.status_code == 200:
        response_json = response.json()
        message = response_json['choices'][0]['message']
        content = message['content']
        return content
    else:
        return ""

# TTS 음성 변환 함수
def request_tts(text):
    file_name = "tts_response_audio.wav"
    endpoint = TTS_ENDPOINT_STEAM
    headers = {
        "Ocp-Apim-Subscription-Key": TTS_APIKEY_STEAM,
        "Content-Type": "application/ssml+xml",
        "X-Microsoft-OutputFormat": "riff-44100hz-16bit-mono-pcm"
    }
    
    body = f"""
            <speak version='1.0' xml:lang='ko-kr'>
            <voice name='ko-KR-JiMinNeural'>
                <prosody rate="20%">
                    {text}
                </prosody>
            </voice>
        </speak>
    """

    response = requests.post(endpoint, headers=headers, data=body)
    if response.status_code == 200:
        with open(file_name, "wb") as audio_file:
            audio_file.write(response.content)
        return file_name
    else:
        return None
    

def clear_function():
    # 초기화할 항목들
    return "", []  # 텍스트박스, 챗봇



# 게임 추천 및 챗봇 대화 흐름
def game_recommendation(game_name, histories):
    app_id = get_exact_app_id(game_name)  # app_id 검색 함수
    
    if app_id is None:
        return "게임을 찾을 수 없습니다.", histories  # 게임이 없으면 반환
    
    # Steam 리뷰 데이터를 가져오기
    URL = f"https://store.steampowered.com/appreviews/{app_id}?json=1&language=english"
    response = requests.get(URL)
    reviews = response.json()
    
    # 리뷰들을 텍스트로 변환 후 요약
    all_reviews_text = " ".join([review['review'] for review in reviews['reviews']])
    summarized_reviews = request_Summarization(all_reviews_text)  # 리뷰 요약

    if summarized_reviews:
        # 감정 분석 수행
        sentiment_analysis_result = request_SentimentAnalysis(text=summarized_reviews)  # 감정 분석 함수
        if sentiment_analysis_result:
            # 감정 분석 결과를 프롬프트에 포함하여 최종 추천 여부를 요청
            prompt = f"이 게임의 감정 분석 결과는 {sentiment_analysis_result}. 이 게임을 추천할까요?"
            return click_gpt_send(game_name, prompt, histories)  # 챗봇 응답 받기
        else:
            return "감정 분석 결과를 가져올 수 없습니다.", histories
    else:
        return "리뷰 요약에 실패했습니다.", histories


# GPT 응답을 처리하고 챗봇의 히스토리를 갱신
def click_gpt_send(game_name, prompt, histories):
    content = request_gpt(prompt)
    histories.append({"role": "user", "content": game_name})  # 사용자 입력 히스토리 추가
    if content:
        histories.append({"role": "assistant", "content": content})  # 챗봇 응답 히스토리 추가
    else:
        histories.append({"role": "assistant", "content": "응답을 받지 못했습니다."})  # 실패 시 응답 추가
    return "", histories



# 챗봇의 응답을 텍스트 정리 후 음성 출력
def change_chatbot(histories):
    history = histories[-1]
    content = history['content']
    # 불필요한 문자(이모티콘, 마크다운 태그 등) 제거 정규식
    cleaned_content = re.sub(r'[^\w\sㄱ-ㅎ가-힣]', '', content)  # 한글, 영어, 숫자, 공백만 남기기
    # 음성 변환 요청
    audio_path = request_tts(cleaned_content)  # 음성 변환
    return audio_path


#### Gradio

In [9]:
with gr.Blocks(theme=theme, css=custom_css) as demo_chat:
    gr.Markdown()
    gr.Markdown("# 🎮오늘 게임 뭐할까?🎮")
    
    with gr.Row():
        with gr.Column(scale=3):
            chatbot = gr.Chatbot(type="messages")  # 챗봇 설정
            with gr.Row():
                prompt = gr.Textbox(label="게임 이름", placeholder="스팀에 있는 게임 이름을 정확히 입력하세요", scale=6)
                send_gpt_button = gr.Button("답변 듣기", scale=1)
                clear_button = gr.Button("초기화")  # 초기화 버튼 아래로 이동
                prompt.submit(game_recommendation, inputs=[prompt, chatbot], outputs=[chatbot, chatbot])
            with gr.Row():
                # 음성 출력 부분
                gpt_audio = gr.Audio(interactive=False, autoplay=True)

    send_gpt_button.click(fn=game_recommendation, inputs=[prompt, chatbot], outputs=[chatbot, chatbot])
    chatbot.change(fn=change_chatbot, inputs=[chatbot], outputs=[gpt_audio])
    # clear 버튼 클릭 시 초기화 동작
    clear_button.click(clear_function, outputs=[prompt, chatbot])

demo_chat.launch()

* Running on local URL:  http://127.0.0.1:7878

To create a public link, set `share=True` in `launch()`.
